# Load Pretrained

In [10]:
import numpy as np
total_data = np.load("data.npy")
total_label = np.load("label.npy")

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(total_data, total_label, test_size=0.33, random_state=42)
X_train_len, X_test_len = len(X_train), len(X_test)
ratio = 1
X_train = X_train[:int(X_train_len * ratio)]
X_test = X_test[:int(X_test_len * ratio)]
y_train = y_train[:int(X_train_len * ratio)]
y_test = y_test[:int(X_test_len * ratio)]
X_train_len, X_test_len = len(X_train), len(X_test)

# CNN
## Model

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.channels = channels

        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)

    def forward(self, x):
        y = F.relu(self.conv1(x))
        y = self.conv2(y)
        return F.relu(x + y)

#实现一个Bottleneck的类，初始化需要输入通道数与GrowthRate这两个参数
class Bottleneck(nn.Module):
    def __init__(self,nChannels,growthRate):
        super(Bottleneck,self).__init__()
    #通常1x1卷积的通道数为GrowthRate的4倍
        interChannels=4*growthRate
        self.bn1=nn.BatchNorm2d(nChannels)
        self.conv1=nn.Conv2d(nChannels,interChannels,kernel_size=1,bias=False)
        self.bn2=nn.BatchNorm2d(interChannels)
        self.conv2=nn.Conv2d(interChannels,growthRate,kernel_size=3,padding=1,bias=False)

    def forward(self,x):
        out=self.conv1(F.relu(self.bn1(x)))
        out=self.conv2(F.relu(self.bn2(out)))
        #将输入x同计算的结果out进行通道拼接
        out=torch.cat((x,out),1)
        return out

class Denseblock(nn.Module):
    def __init__(self,nChannels,growthRate,nDenseBlocks):
        super(Denseblock,self).__init__()
        layers=[]
        #将每一个Bottleneck利用nn.Sequential()整合起来，输入通道数需要线性增长
        for i in range(int(nDenseBlocks)):
            layers.append(Bottleneck(nChannels,growthRate))
            nChannels+=growthRate
        self.denseblock=nn.Sequential(*layers)
    def forward(self,x):
        return self.denseblock(x)


class CnnModel(nn.Module):
    def __init__(self):
        super(CnnModel, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        # self.conv3 = nn.Sequential(
        #     nn.Conv2d(32, 64, kernel_size=5),
        #     nn.BatchNorm2d(64),
        #     nn.ReLU()
        # )
        self.mp = nn.MaxPool2d(4)

        self.rblock1 = ResidualBlock(16)
        self.rblock2 = ResidualBlock(32)
        # self.rblock3 = ResidualBlock(64)

        self.fc = nn.Sequential(
            nn.Linear(128, 10)
        )
        self._set_init(self.conv1)
        self._set_init(self.conv2)
        # self._set_init(self.conv3)
        self._set_init(self.fc)

    def _set_init(self, sequence: nn.Sequential):  # 参数初始化
        for name, param in sequence.named_parameters():
            if 'weight' in name:
                init.normal_(param, 0, 1.0)
            if 'bias' in name:
                init.constant_(param, 0.5)

    def forward(self, x):
        # Flatten data from (n,1,28,28) to (n,784)
        in_size = x.size(0)
        x = self.mp(self.conv1(x))
        x = self.rblock1(x)
        x = self.mp(self.conv2(x))
        x = self.rblock2(x)
        # x = self.mp(self.conv3(x))
        # x = self.rblock3(x)
        # x = self.mp(x)
        # x = self.mp(F.relu(self.conv3(x)))
        # x = self.rblock3(x)
        x = x.view(in_size, -1)  # flatten
        #         print(x.size(1))
        return self.fc(x)


_model = CnnModel().float().cuda()

_test_input = torch.rand((1, 1, 56, 56)).cuda()
_model(_test_input)

tensor([[  1.1844,  -8.3257,  -0.5414, -55.2046,   7.1606, -33.5012,  31.8390,
         -18.3668, -35.7643,  10.0661]], device='cuda:0',
       grad_fn=<AddmmBackward>)

In [85]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict


class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super(_DenseLayer, self).__init__()
        self.add_module('norm1', nn.BatchNorm2d(num_input_features)),
        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv2d(num_input_features, bn_size *
                                           growth_rate, kernel_size=1, stride=1, bias=False)),
        self.add_module('norm2', nn.BatchNorm2d(bn_size * growth_rate)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv2d(bn_size * growth_rate, growth_rate,
                                           kernel_size=3, stride=1, padding=1, bias=False)),
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super(_DenseLayer, self).forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)


class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate, growth_rate, bn_size, drop_rate)
            self.add_module('denselayer%d' % (i + 1), layer)


class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2))


class DenseNet(nn.Module):
    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0, num_classes=1000):

        super(DenseNet, self).__init__()

        # First convolution
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(1, num_init_features, kernel_size=3, bias=False)),
            ('norm0', nn.BatchNorm2d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool2d(kernel_size=2)),
        ]))

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(num_layers=num_layers, num_input_features=num_features,
                                bn_size=bn_size, growth_rate=growth_rate, drop_rate=drop_rate)
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features, num_output_features=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm2d(num_features))

        # Linear layer
        num_features = 1150
        self.classifier = nn.Linear(num_features, num_classes)

        # Official init from torch repo.
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight.data)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.avg_pool2d(out, kernel_size=2, stride=1).view(features.size(0), -1)
        # print(out.shape)
        out = self.classifier(out)
        return out


net = DenseNet(num_init_features=1, growth_rate=4, block_config=(6, 12, 4), num_classes=10)
# print(net)
_model = net.float().cuda()

_test_input = torch.rand((1, 1, 56, 56)).cuda()
_model(_test_input)

tensor([[ 0.1628, -0.0028, -0.2856,  0.0635, -0.0646,  0.3127, -0.2245,  0.5216,
          0.2237, -0.2657]], device='cuda:0', grad_fn=<AddmmBackward>)


## Prepare Train Data

In [95]:

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.utils.data as Data

writer = SummaryWriter('runs/train')


# class FreqData(Dataset):
#     def __len__(self):
#         return len(X_train)
#
#     def __getitem__(self, index):
#         return X_train[index, :, :], y_train[index]


In [90]:

bs = 100
batch = 0

In [92]:

_loss_fn = nn.CrossEntropyLoss()
# _optimizer = torch.optim.SGD(_model.parameters(), lr=1e-4)
_optimizer = torch.optim.AdamW(_model.parameters(), lr=1e-4)

In [68]:

def np2ts(a, l):
    a = torch.Tensor(a)
    a = a.unsqueeze(1)  # used for CNN data
    l = torch.Tensor(l).long()
    return a, l

train_dataset = Data.TensorDataset(*np2ts(X_train, y_train))
train_loader = Data.DataLoader(
    dataset=train_dataset,
    batch_size=bs,
    shuffle=True,
)

test_dataset = Data.TensorDataset(*np2ts(X_test, y_test))
test_loader = Data.DataLoader(
    dataset=test_dataset,
    batch_size=bs,
    shuffle=True,
)

def to_cuda(*ts: torch.Tensor):
    return [_ts.cuda() for _ts in ts]



## Evaluation

In [69]:

def eval():
    accuracy = []
    _model.eval()
    for stp, (test_x, test_y) in enumerate(test_loader):
        test_x, test_y = to_cuda(test_x, test_y)
        test_output = _model(test_x)
        _, pred_y = torch.max(test_output, 1)
        accuracy.append(torch.sum(pred_y == test_y).cpu().item() / len(test_y))
    score = np.mean(accuracy)
    return score



## Train

In [100]:
max_val_score = 0


In [115]:

is_break = False
while True:
    if is_break:
        break
    for b, (_X, _y) in enumerate(train_loader):
        _model.train()
        _X = _X.float()  #.cuda()
        _y = _y.long()  #.cuda()
        _X, _y = to_cuda(_X, _y)
        pred = _model(_X)
        loss = _loss_fn(pred, _y)

        # BP
        _optimizer.zero_grad()
        loss.backward()
        _optimizer.step()
        batch += 1

        if batch % 50 == 0:
            _model.eval()
            eval_score = eval()
            print(f"{batch}, {loss} <-> {eval_score}")
            writer.add_scalars('CNN Training Loss',
                               {'Train': loss, 'Val': eval_score},
                               batch)
            if eval_score > max_val_score:
                torch.save(_model.state_dict(), f"cnn-model-best.pth")
                print(f"best model at {batch}")
                max_val_score = eval_score
        if batch % 400 == 0:
            is_break = True
            break

8450, 0.22914332151412964 <-> 0.8659493670886076
best model at 8450
8500, 0.22425487637519836 <-> 0.8648101265822786
8550, 0.11168172210454941 <-> 0.8640506329113925
8600, 0.17536665499210358 <-> 0.8621518987341772
8650, 0.22484250366687775 <-> 0.8659493670886077
best model at 8650
8700, 0.2538182735443115 <-> 0.8675949367088607
best model at 8700
8750, 0.15746936202049255 <-> 0.8664556962025317
8800, 0.3418591320514679 <-> 0.8639240506329116


In [9]:
torch.save(_model.state_dict(), f"cnn-model-{batch}.pth")

AttributeError: 'list' object has no attribute 'float'